In [1]:
import pandas as pd
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
df_train = pd.read_csv('../../datasets/Titanic/train.csv')
df_test=pd.read_csv('../../datasets/Titanic/test.csv')

In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df_train['Age'].fillna(value=df_train['Age'].mean(),inplace=True)

In [6]:
df_train['Embarked'].mode().iloc[0]

'S'

In [7]:
df_train['Embarked'].fillna(value=df_train['Embarked'].mode().iloc[0],inplace=True)

In [8]:
df_train['Age_cat']=df_train['Age'].apply(lambda age:'child' if age<15 else 'old' if age>55 else 'young')
df_train['Family_size']=df_train['Parch']+df_train['SibSp']
df_train['iscabin']=(~df_train['Cabin'].isna()).apply(int)

In [9]:
df_train[['C','Q','S']]=pd.get_dummies(df_train['Embarked'],dtype=int)
df_train['Sex_enc']=df_train['Sex'].map({'male':0,'female':1})
df_train['Age_cat_enc']=df_train['Age_cat'].map({'child':0,'young':1,'old':2})

In [10]:
column_num=['Fare','Family_size']

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(df_train[column_num])
df_train[column_num]=scaler.transform(df_train[column_num])

In [12]:
from sklearn.svm import SVC

In [13]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Age_cat',
       'Family_size', 'iscabin', 'C', 'Q', 'S', 'Sex_enc', 'Age_cat_enc'],
      dtype='object')

In [14]:
selected_columns=['Pclass', 'Fare', 'Family_size', 'iscabin',  'Sex_enc', 'Age_cat_enc','C','Q','S']

In [15]:
df_train[selected_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Pclass       891 non-null    int64  
 1   Fare         891 non-null    float64
 2   Family_size  891 non-null    float64
 3   iscabin      891 non-null    int64  
 4   Sex_enc      891 non-null    int64  
 5   Age_cat_enc  891 non-null    int64  
 6   C            891 non-null    int32  
 7   Q            891 non-null    int32  
 8   S            891 non-null    int32  
dtypes: float64(2), int32(3), int64(4)
memory usage: 52.3 KB


In [16]:
X=df_train[selected_columns]
y=df_train['Survived']

In [17]:
model=SVC(kernel='rbf')
model.fit(X,y)
model.score(X,y)

0.8282828282828283

In [18]:
df_test['Age'].fillna(value=df_train['Age'].mean(),inplace=True)
df_test['Fare'].fillna(value=df_train['Fare'].mean(),inplace=True)

In [19]:
df_test['Age_cat']=df_train['Age'].apply(lambda age:'child' if age<15 else 'old' if age>55 else 'young')
df_test['Family_size']=df_test['Parch']+df_test['SibSp']
df_test['iscabin']=(~df_test['Cabin'].isna()).apply(int)

In [20]:
df_test[['C','Q','S']]=pd.get_dummies(df_test['Embarked'],dtype=int)
df_test['Sex_enc']=df_test['Sex'].map({'male':0,'female':1})
df_test['Age_cat_enc']=df_test['Age_cat'].map({'child':0,'young':1,'old':2})

In [21]:
df_test[column_num]=scaler.transform(df_test[column_num])

In [22]:
X_test=df_test[selected_columns]

In [23]:
yp=model.predict(X_test)

In [24]:
df_submit=pd.DataFrame({'PassengerId':df_test['PassengerId'],'Survived':yp})
df_submit.to_csv('sub3.csv',index=False)

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
model1=RandomForestClassifier(random_state=1)

In [27]:
model1.fit(X,y)
model1.score(X,y)

0.9371492704826038

In [28]:
yp1=model1.predict(X_test)

In [29]:
df_submit=pd.DataFrame({'PassengerId':df_test['PassengerId'],'Survived':yp1})
df_submit.to_csv('sub3.csv',index=False)

In [30]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
d1 = {
    'n_estimators': [10, 25, 50, 100, 150, 200],  # Number of weak learners
    'learning_rate': [0.01, 0.1, 0.5, 1.0],  # Weight applied to each classifier at each boosting iteration
    'algorithm': ['SAMME', 'SAMME.R'],  # Algorithm for boosting (SAMME or SAMME.R)
    'random_state': [42, None]  # Control the random seed
}

grid_cv=GridSearchCV(estimator=AdaBoostClassifier(), param_grid=d1, cv=5, n_jobs=4)
grid_cv.fit(X,y)

GridSearchCV(cv=5, estimator=AdaBoostClassifier(), n_jobs=4,
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [0.01, 0.1, 0.5, 1.0],
                         'n_estimators': [10, 25, 50, 100, 150, 200],
                         'random_state': [42, None]})

In [33]:
grid_cv.best_params_

{'algorithm': 'SAMME',
 'learning_rate': 0.5,
 'n_estimators': 25,
 'random_state': 42}

In [35]:
model2=AdaBoostClassifier(estimator=DecisionTreeClassifier(),algorithm= 'SAMME',
 learning_rate= 1.0,
 n_estimators=25,
 random_state= 1)

In [36]:
model2.fit(X,y)

AdaBoostClassifier(algorithm='SAMME', estimator=DecisionTreeClassifier(),
                   n_estimators=25, random_state=1)

In [37]:
yp2=model2.predict(X_test)

In [38]:
df_submit=pd.DataFrame({'PassengerId':df_test['PassengerId'],'Survived':yp2})
#df_submit.to_csv('sub4.csv',index=False)

In [39]:
model2.score(X,y)

0.9371492704826038

In [40]:
d1={'n_estimators':[10,25,50,60,70]
    ,'max_depth':[4,5,6,]
    ,'min_samples_leaf':[2,3,4,5]}
grid_cv=GridSearchCV(estimator=RandomForestClassifier(random_state=1), param_grid=d1, cv=5, n_jobs=4)
grid_cv.fit(X,y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=1), n_jobs=4,
             param_grid={'max_depth': [4, 5, 6],
                         'min_samples_leaf': [2, 3, 4, 5],
                         'n_estimators': [10, 25, 50, 60, 70]})

In [41]:
grid_cv.best_params_

{'max_depth': 6, 'min_samples_leaf': 2, 'n_estimators': 25}

In [42]:
model_fi=RandomForestClassifier(max_depth= 6, min_samples_leaf= 4, n_estimators= 55)
model_fi.fit(X,y)

RandomForestClassifier(max_depth=6, min_samples_leaf=4, n_estimators=55)

In [43]:
yp6=model_fi.predict(X_test)

In [46]:
df_submit=pd.DataFrame({'PassengerId':df_test['PassengerId'],'Survived':yp6})
#df_submit.to_csv('sub10.csv',index=False)

In [47]:
df_submit

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [48]:
from sklearn.neighbors import KNeighborsClassifier
model6=KNeighborsClassifier(n_neighbors=9)
model6.fit(X,y)

KNeighborsClassifier(n_neighbors=9)

In [50]:
yp7=model6.predict(X_test)

In [53]:
df_submit1=pd.DataFrame({'PassengerId':df_test['PassengerId'],'Survived':yp7})

In [54]:
df_submit1

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
